In [1]:
from scrapfly import ScrapflyClient, ScrapeConfig, ScrapflyError
 
scrapfly = ScrapflyClient(key="scp-live-01fc8534987042f4a2fe553c5cf6a2df", max_concurrency=2)
url = 'https://www.officedepot.com/a/products/3587544'
listing_result = scrapfly.scrape(ScrapeConfig(url=url, render_js=True, country="US", asp=True, retry=False, rendering_wait=10000))
listing_html = listing_result.scrape_result['content']

In [2]:
from bs4 import BeautifulSoup
import re
import json
def extract_json_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    script_tags = soup.find_all('script')

    json_data_list = []

    for script in script_tags:
        script_content = script.string
        if script_content:
            # Use a regular expression to find JSON-like content
            json_pattern = re.compile(r'({.*})', re.DOTALL)
            matches = json_pattern.findall(script_content)
            for match in matches:
                try:
                    json_data = json.loads(match)
                    json_data_list.append(json_data)
                except json.JSONDecodeError:
                    pass  # Ignore invalid JSON

    return json_data_list

json_data_list = extract_json_from_html(listing_html)

In [36]:
item5_data = json_data_list[5]

In [53]:
image_url = item5_data['MasterTmsUdo']['prod_img']
category_id = item5_data['MasterTmsUdo']['category_id']

In [49]:
data = json_data_list[6]

In [56]:
extracted_data = {
    'ProductName': data.get('name'),
    'sku': data.get('sku'),
    'description': data.get('description'),
    'category_name': data['breadcrumbs'][2]['name'] if len(data['breadcrumbs']) > 2 else None,
    'category_id' : category_id,
    'brand': data.get('brand'),
    'color': data.get('color'),
    'sub_category_url': data['breadcrumbs'][3]['url'] if len(data['breadcrumbs']) > 3 else None,
    'image_content_url': data['image'].get('contentUrl'),
    'price_currency': data['offers'].get('priceCurrency'),
    'price': data['offers'].get('price'),
    'item_condition': data['offers'].get('itemCondition'),
    'availability': data['offers'].get('availability'),
    'seller_name': data['offers']['seller'].get('name'),
    'mpn': data.get('mpn'),
    'image_url':image_url
}

# Print the extracted data
print(json.dumps(extracted_data, indent=4))

{
    "ProductName": "ASUS Vivobook Go E1504FA-OS36 Laptop, 15.6&quot; Screen, AMD Ryzen 3, AI Enabled, 8GB Memory, 512GB Solid State Drive, Windows&reg; 11 Home,",
    "sku": "3587544",
    "description": "Choose this ASUS Vivobook Go AI Enabled laptop to elevate your tech game to outstanding new heights. Embrace a future where cutting-edge technology meets human-like intelligence&mdash;exploring territories such as language comprehension, image recognition, and independent decision-making. This technology wonder doesn't just perform tasks; it evolves, learning from every interaction to serve you better with each use. Engage in seamless communication, whether through voice or text, and witness the magic of technology that 'listens' and 'understands' your needs. With the ability to interpret visuals without human sight, your laptop surpasses boundaries, offering insights and predictions by connecting the dots from your digital world. Imagine a world where your laptop isn't just a tool,